In [1]:
# Constantes 

# Masse atomique : 
masses = {"N": 14, "C": 12, "CA": 12,"O": 16,"S": 12}

# caracteres des aa : 
hydrophobic = ["F" , "G" , "I" , "L" , "M" , "V" , "W" , "Y" ]
hydrophile = [ "A" , "C" , "D" , "E" , "H" , "K" , "N" , "P" , "Q" , "R" , "S" , "T" ]

In [2]:
#Packages 
from Bio.PDB import *
from Bio.PDB.DSSP import DSSP
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [3]:
# calcul du centre de masse et extraction des carbonnes alpha
def manipulation_PDB(nom_pdb): 
    parser = PDBParser() 
    structure = parser.get_structure("PHA-L", nom_pdb)

    # initialisation des variables 
    all_CA = [] # listes des CA de la proteine

    # coordonnes du centre de masse 
    xnum = 0
    xden = 0
    ynum = 0
    yden = 0
    znum = 0
    zden = 0

    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    #print(f"Atom Name: {atom.name} - Atom Type: {atom.element} Residue Number: {residue.id[1]} {atom.coord[1]} {residue.resname}")
                    xi = f"{atom.coord[0]:.3f}" # coordonnee x de l'atome i
                    yi = f"{atom.coord[1]:.3f}" # coordonnee y de l'atome i
                    zi = f"{atom.coord[2]:.3f}" # coordonne z de l'atome i
                    mi = masses[atom.element]
                    xnum += (mi * float(xi))
                    xden += mi
                    ynum += (mi * float(yi))
                    yden += mi
                    znum += (mi * float(zi))
                    zden += mi
                    if atom.name == 'CA' :
                        x_coord = atom.coord[0]
                        y_coord = atom.coord[1]
                        z_coord = atom.coord[2]
                        all_CA.append([residue.resname,residue.id[1],x_coord,y_coord,z_coord])
    # calcul centre de masse 
    xg = xnum /xden
    yg = ynum /yden
    zg = znum /zden
    center_of_mass = [ xg , yg , zg ]

    # calcul de l'accessibilite au solvant et ajout pour chaque residu
    dssp = DSSP(structure[0],nom_pdb, dssp='mkdssp')
    for i in range (0,len(all_CA)):
        a_key = list(dssp.keys())[i]
        #print(dssp[a_key][3])
        if dssp[a_key][3] > 0.3 : 
            all_CA[i].append("A")
            all_CA[i].append(dssp[a_key][1])
        else : 
            all_CA[i].append("E")
            all_CA[i].append(dssp[a_key][1])
            
    outpout_pdb = []
    outpout_pdb.append(center_of_mass)
    outpout_pdb.append(all_CA)
    
    return (outpout_pdb)
    

In [50]:
out_1 = manipulation_PDB("protein1_only.pdb")

In [51]:
center_of_mass = out_1[0]
all_CA = out_1[1]

In [52]:
# recupere les accessible 

 

In [53]:
liste_acces = sep_access_aa(all_CA)

In [54]:
liste_acces

[['MET', 1, 5.353, 2.208, 75.28, 'A', 'M', 'hphobic'],
 ['ASN', 2, 4.346, 5.591, 76.723, 'A', 'N', 'hphile'],
 ['THR', 4, -1.234, 3.199, 73.026, 'A', 'T', 'hphile'],
 ['GLU', 5, -1.825, 3.269, 69.25, 'A', 'E', 'hphile'],
 ['PRO', 7, -5.655, 2.886, 63.078, 'A', 'P', 'hphile'],
 ['ASN', 8, -7.692, 5.999, 63.9, 'A', 'N', 'hphile'],
 ['PHE', 13, -1.827, 6.111, 78.481, 'A', 'F', 'hphobic'],
 ['ASN', 15, -1.953, -0.448, 78.445, 'A', 'N', 'hphile'],
 ['LYS', 16, 0.749, -1.879, 80.75, 'A', 'K', 'hphile'],
 ['THR', 17, -2.053, -1.455, 83.307, 'A', 'T', 'hphile'],
 ['GLY', 18, -4.331, -3.605, 81.088, 'A', 'G', 'hphobic'],
 ['VAL', 19, -7.167, -1.066, 80.887, 'A', 'V', 'hphobic'],
 ['ARG', 21, -9.104, 0.97, 74.879, 'A', 'R', 'hphile'],
 ['GLU', 25, -17.163, 3.232, 72.023, 'A', 'E', 'hphile'],
 ['PRO', 27, -15.297, 6.332, 77.94, 'A', 'P', 'hphile'],
 ['TYR', 29, -12.079, 7.62, 82.436, 'A', 'Y', 'hphobic'],
 ['GLU', 33, -10.644, 11.881, 87.354, 'A', 'E', 'hphile'],
 ['PRO', 34, -14.416, 11.964, 86.

In [16]:
# recupere les accessible 

def hydro_all(liste_CA): 
    hydrophobic = ["F" , "G" , "I" , "L" , "M" , "V" , "W" , "Y" ]
    hydrophile = [ "A" , "C" , "D" , "E" , "H" , "K" , "N" , "P" , "Q" , "R" , "S" , "T" ]
    hydrophobe_count = 0 
    for i in range (0,len(liste_CA)):
        if liste_CA[i][6] in hydrophobic : 
            hydrophobe_count += 1 
    print(hydrophobe_count)
    return(hydrophobe_count)

In [17]:
hydrophobe_tot =  hydro_all(all_CA)

170


In [18]:
# calcul total hydrophobe et hydrophile 

def hydro_count(liste_accessible):
    tot_hydrophile_exp = 0
    tot_hydrophob_exp = 0 
    for i in range (0,len (liste_accessible)):
        if liste_accessible[i][7] == "hphobic" : 
            tot_hydrophob_exp += 1
        else : 
            tot_hydrophile_exp += 1 
    totaux = []
    totaux.append(tot_hydrophile_exp)
    totaux.append(tot_hydrophob_exp)
    
    return totaux

In [56]:
totaux= hydro_count(liste_acces)

In [25]:
totaux_count

[70, 78]

In [33]:
# calcul points demi-sphere

def saff_kuijlaars_half_sphere(num_points, radius,centre_de_masse):
    points = []
    phi_0 = 0
    for k in range(2, num_points//2):  # Utiliser num_points // 2 pour obtenir la moitié supérieure
        hk = -1 + ((2 * (k - 1)) / (num_points - 1))
        theta = np.arccos(hk)
        phi = (phi_0 + ((3.6) / (num_points) ** (1 / 2)) * (1 / (1 - hk ** 2) ** (1 / 2)))% (2*3.14)
        phi_0 = phi
        x = radius * np.sin(theta) * np.cos(phi)
        y = radius * np.sin(theta) * np.sin(phi)
        z = radius * np.cos(theta)
        points.append([x, y, z])

    half_sphere_points_centered = [np.array(point) + np.array(centre_de_masse) for point in points]
        
    return half_sphere_points_centered


num_points = 100+2  # Nombre de points sur la demi-sphère
radius = 100.0  # Rayon de la demi-sphère

In [34]:
sphere_points = saff_kuijlaars_half_sphere(num_points,radius,center_of_mass)

In [32]:
# calcul des vecteurs normaux à chaque plan 

def calc_vecteur_normal(array_coord_pts,c_mass):
    coord_vnorm=[]
    for i in range(0,len(array_coord_pts)):
        x_vnor = c_mass[0]-array_coord_pts[i][0]
        y_vnor = c_mass[1]-array_coord_pts[i][1]
        z_vnor = c_mass[2]-array_coord_pts[i][2]
        niem_vnorm = [x_vnor,y_vnor, z_vnor]
        coord_vnorm.append(niem_vnorm)           
    return coord_vnorm

In [44]:
normal_vector = calc_vecteur_normal(sphere_points,center_of_mass)

In [59]:
normal_vector[0]

[4.500752860094019, -19.283714462082372, 98.01980198019803]

In [61]:
# calcul des plans
def calcul_eq_d_plan_start(vect_nor,c_mass):
    cartesian_equation = []
    for i in range (0,len(vect_nor)): 
        d=vect_nor[i][0]*c_mass[0]+vect_nor[i][1]*c_mass[1]+vect_nor[i][2]*c_mass[2]
        d=round(-1*d,3)
        cartesian_equation.append([[d+7.5,d-7.5],[0,0,0,0]])
    return cartesian_equation

In [62]:
cartesian_eq = calcul_eq_d_plan_start(normal_vector,center_of_mass)

In [63]:
cartesian_eq 

[[[-5919.096, -5934.096], [0, 0, 0, 0]],
 [[-6114.812, -6129.812], [0, 0, 0, 0]],
 [[-6970.452, -6985.452], [0, 0, 0, 0]],
 [[-7368.858, -7383.858], [0, 0, 0, 0]],
 [[-6888.711, -6903.711], [0, 0, 0, 0]],
 [[-5791.98, -5806.98], [0, 0, 0, 0]],
 [[-4646.164, -4661.164], [0, 0, 0, 0]],
 [[-3935.743, -3950.743], [0, 0, 0, 0]],
 [[-3889.65, -3904.65], [0, 0, 0, 0]],
 [[-4451.958, -4466.958], [0, 0, 0, 0]],
 [[-5361.64, -5376.64], [0, 0, 0, 0]],
 [[-6274.906, -6289.906], [0, 0, 0, 0]],
 [[-6881.056, -6896.056], [0, 0, 0, 0]],
 [[-6982.876, -6997.876], [0, 0, 0, 0]],
 [[-6531.188, -6546.188], [0, 0, 0, 0]],
 [[-5617.014, -5632.014], [0, 0, 0, 0]],
 [[-4425.553, -4440.553], [0, 0, 0, 0]],
 [[-3212.469, -3227.469], [0, 0, 0, 0]],
 [[-2202.648, -2217.648], [0, 0, 0, 0]],
 [[-1572.546, -1587.546], [0, 0, 0, 0]],
 [[-1414.371, -1429.371], [0, 0, 0, 0]],
 [[-1726.054, -1741.054], [0, 0, 0, 0]],
 [[-2419.32, -2434.32], [0, 0, 0, 0]],
 [[-3341.858, -3356.858], [0, 0, 0, 0]],
 [[-4308.022, -4323.022]

In [72]:
# fonction necessaire 

# verification position membrane par rapport proteine
def calcul_pos_memb(vect_nor,accessible_aa,d_plan_h,d_plan_b):
    id_aa = []
    position_memb = "FALSE"
    for i in range (0,len(accessible_aa)):
        dhaut = vect_nor[0]*accessible_aa[i][2]+vect_nor[1]*accessible_aa[i][3]+vect_nor[2]*accessible_aa[i][3]+d_plan_h
        dbas = vect_nor[0]*accessible_aa[i][2]+vect_nor[1]*accessible_aa[i][3]+vect_nor[2]*accessible_aa[i][3]+d_plan_b
        if dhaut <= 0 and dbas >= 0 : 
            position_memb = "TRUE"
            id_aa.append(i)
        elif dhaut >= 0 and dbas <= 0 : 
            position_memb = "TRUE"
            id_aa.append(i)
    return (position_memb,id_aa)


# calcul de l'hydrophobicite relative 

def calcul_hydrophobicite_rel(list_accessible_aa,id_aa,totaux):
    hydrophobe_memb = 0
    hydrophile_memb = 0
    for i in id_aa : 
        if list_accessible_aa[i][7] == "hphobic" : 
            hydrophobe_memb += 1
        elif list_accessible_aa[i][7] == "hphilie" :
            hydrophile_memb += 1
    hphile_exp_hors_memb = (hydrophobe_tot - hydrophile_memb)
    relative_hydrophobicity = ((hphile_exp_hors_memb /totaux[0]) + (hydrophobe_memb/hydrophobe_tot))
    return relative_hydrophobicity

In [77]:
#calcul du max hydrophobe plan 

def max_hydroph(list_norm_vect,eq_cartesian1,liste_accessible,totaux):
    eq_cartesian = eq_cartesian1.copy()
    for i in range (0 , len(list_norm_vect)):
        print(i)
        tranche_haut = "TRUE"
        tranche_bas  = "TRUE"
        d_haut_up = eq_cartesian[i][0][0]
        d_haut_down = eq_cartesian[i][0][0]
        d_bas_up = eq_cartesian[i][0][1]
        d_bas_down = eq_cartesian[i][0][1]
        while tranche_haut == "TRUE" : 
            plan_haut = calcul_pos_memb(list_norm_vect[i],liste_accessible,d_haut_up,d_bas_up)
            tranche_haut = plan_haut[0] # TRUE si plan sur proteine sinon false
            id_aa_memb = plan_haut[1] # id liste des residu entre les deux plans 
            if tranche_haut == "TRUE" : 
                rel_hydrophobie  = calcul_hydrophobicite_rel(liste_accessible,id_aa_memb,totaux)
                if rel_hydrophobie > eq_cartesian[i][1][2]:
                    print("haut",rel_hydrophobie)
                    eq_cartesian[i][1][2] = rel_hydrophobie
                    eq_cartesian[i][1][0] = d_haut_up
                    eq_cartesian[i][1][1] = d_bas_up
                    d_haut_up += 1 
                    d_bas_up += 1
                else : 
                    #print(cartesian_eq_d_start[i][3][2])
                    d_haut_up += 1 
                    d_bas_up += 1             
        while tranche_bas == "TRUE":
            plan_bas = calcul_pos_memb(list_norm_vect[i],liste_accessible,d_haut_down,d_bas_down)
            tranche_bas = plan_bas[0] # TRUE si plan sur proteine sinon FALSE
            id_aa_memb_bas = plan_bas[1]
            if tranche_bas == "TRUE":
                rel_hydrophobie_bas  = calcul_hydrophobicite_rel(liste_accessible,id_aa_memb_bas,totaux)
                if rel_hydrophobie_bas > eq_cartesian[i][1][2]:
                    print("bas",rel_hydrophobie_bas)
                    eq_cartesian[i][1][2] = rel_hydrophobie_bas
                    eq_cartesian[i][1][0] = d_haut_down
                    eq_cartesian[i][1][1] = d_bas_down 
                    d_haut_down -= 1 
                    d_bas_down -= 1
                else :
                    d_haut_down -= 1 
                    d_bas_down -= 1
    return (eq_cartesian)

In [78]:
le_mieux = max_hydroph(normal_vector,cartesian_eq,liste_acces,totaux)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [79]:
# meilleurs plans 

def best_plane(list_best_4_each,list_nvector):
    best_phobe_value = []
    # recupere les valeurs d'hydrophobicite pour chaque axe
    for i in range(0,len(list_best_4_each)):
        best_phobe_value.append(list_best_4_each[i][1][2])
    Best_v_num = 0
    list_best = []
    # cherche les plans avec la meilleurs valeurs (plusieurs plan si valeurs identiques) 
    for i in range(0,len(best_phobe_value)):
        if best_phobe_value[i] >= Best_v_num :
            Best_v_num = best_phobe_value[i]
    for i in range(0,len(best_phobe_value)):
        if best_phobe_value[i] >= Best_v_num :
            list_best.append(i)
            print(i)
    # recuperer l'equation des meilleurs plan et la valeur d'hydrophobicite associe 
    best_planes = []
    for i in range (0,len(list_best)):
        ieme_id = list_best[i]
        ax= list_nvector[ieme_id][0]
        by= list_nvector[ieme_id][1]
        cz= list_nvector[ieme_id][2]
        dh= list_best_4_each[ieme_id][1][0]
        db= list_best_4_each[ieme_id][1][1]
        val_memb_hphobe = list_best_4_each[ieme_id][1][2]
        best_planes.append([ax,by,cz,dh,db,val_memb_hphobe])
    return best_planes

In [80]:
les_mieux = best_plane(le_mieux,normal_vector)

25
32


In [83]:
def maximise_hydrophobicite_plane(list_best_planes1,totaux,accessible_aa):
    list_best_planes = list_best_planes1.copy()
    if len(list_best_planes) == 1 : 
        dhaut = list_best_planes[0][3] 
        dbas  = list_best_planes[0][4]
        dhaut += 0.5
        dbas -= 0.5
        norm_vector = list_best_planes[0][0:3] 
        limite = 10
        
        while limite != 0 : 
            tranche = calcul_pos_memb(norm_vector,accessible_aa,dhaut,dbas)
            id_aa_memb = tranche[1]
            rel_hydrophobie  = calcul_hydrophobicite_rel(accessible_aa,id_aa_memb,totaux)
            print(rel_hydrophobie)
            if rel_hydrophobie > list_best_planes[0][5]:
                limite = 10 
                print(f" value : {rel_hydrophobie}")
                list_best_planes[0][5] = rel_hydrophobie
                list_best_planes[0][3] = dhaut
                list_best_planes[0][4] = dbas
                dhaut += 0.5
                dbas -= 0.5
            else :
                limite -= 1
                dhaut  += 0.5
                dbas   -= 0.5
    else : 
        for i in range(0,len(list_best_planes)):
            dhaut  = list_best_planes[i][3] 
            dbas   = list_best_planes[i][4]
            dhaut += 0.5
            dbas  += 0.5
            norm_vector = list_best_planes[i][0:3] 
            limite = 10
            while limite != 0 : 
                tranche = calcul_pos_memb(norm_vector,accessible_aa,dhaut,dbas)
                id_aa_memb = tranche[1]
                rel_hydrophobie  = calcul_hydrophobicite_rel(accessible_aa,id_aa_memb,totaux)
                print(rel_hydrophobie)
                if rel_hydrophobie > list_best_planes[i][5]:
                    limite = 10 
                    print(f" value : {rel_hydrophobie}")
                    list_best_planes[i][5] = rel_hydrophobie
                    list_best_planes[i][3] = dhaut
                    list_best_planes[i][4] = dbas
                    dhaut += 0.5
                    dbas -= 0.5
                else :
                    limite -= 1
                    dhaut  += 0.5
                    dbas   -= 0.5
                    
        best_phobe_value = []
        for i in range(0,len(list_best_planes)):
            best_phobe_value.append((list_best_planes[i][5]))
        Best_v_num = 0
        list_best = [] 
        for i in range(0,len(best_phobe_value)):
            if best_phobe_value[i] >= Best_v_num :
                Best_v_num = best_phobe_value[i]
                print(Best_v_num)
        for i in range(0,len(best_phobe_value)):
            if best_phobe_value[i] >= Best_v_num :
                list_best.append(i)
        # recuperer l'equation des meilleurs plan et la valeur d'hydrophobicite associe 
        best_planes = []
        for i  in range (0, len(list_best)) :
            ido = list_best[i]
            best_planes.append(list_best_planes[ido])
            print(list_best[i])
        list_best_planes = best_planes
    return list_best_planes
    

In [85]:
alors = maximise_hydrophobicite_plane(les_mieux,totaux,liste_acces)

2.434453781512605
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
2.4403361344537813
0
1


In [87]:
alors

[[44.12113261390985,
  75.49592599763092,
  48.51485148514851,
  -5138.035,
  -5153.035,
  2.4403361344537813],
 [-75.3594679083776,
  -55.85774732338147,
  34.653465346534645,
  -1273.86,
  -1288.86,
  2.4403361344537813]]

In [191]:
import numpy as np

def membrane_represention (liste_plan,centre_de_masse) : 
    for i in range (0,len(liste_plan)) :
        ax = liste_plan[i][0]
        by = liste_plan[i][1]
        cz = liste_plan[i][2]
        d1 = liste_plan[i][3]
        d2 = liste_plan[i][4]
        plane1 = {'A': ax , 'B': by, 'C': cz , 'D': d1 }
        plane2 = {'A': ax , 'B': by, 'C': cz , 'D': d2 }
    #Centre de masse de la protéine
    # Générez des points dans la région entre les deux plans
        points = []
                x, y = np.meshgrid(range(20), range(20))
                z1 = (-plane1['D'] - plane1['A'] * x - plane1['B'] * y) / plane1['C']
                z2 = (-plane2['D'] - plane2['A'] * x - plane2['B'] * y) / plane2['C']
                points.append([x, y, z1])
                points.append([x, y, z2])

    # Décalez les points pour qu'ils soient centrés autour du centre de masse
        points_centered = [point + centre_de_masse for point in points]

    # Exportez les coordonnées des points vers un fichier PDB
        nom_fichier = f"membrane_representation{i}.pdb"
        with open(nom_fichier, 'w') as f:
            for i, point in enumerate(points_centered):
                f.write(f"ATOM  {i+1:4}  CA  MEM M   1    {point[0]:8.3f}{point[1]:8.3f}{point[2]:8.3f}\n")

        print(f"Points générés et enregistrés dans '{nom_fichier}")

In [199]:
x, y = np.meshgrid(range(20), range(20))

In [16]:
## programme principal

out_pdb = manipulation_PDB("protein1_only.pdb")
centre_de_masse= out_pdb[0]
all_CA = out_pdb[1]


In [17]:
out_pdb

[[-8.681831934831866, 30.219536473326407, 66.80707300790084],
 [['MET', 1, 5.353, 2.208, 75.28, 'A', 'M'],
  ['ASN', 2, 4.346, 5.591, 76.723, 'A', 'N'],
  ['GLY', 3, 0.832, 5.515, 75.239, 'E', 'G'],
  ['THR', 4, -1.234, 3.199, 73.026, 'A', 'T'],
  ['GLU', 5, -1.825, 3.269, 69.25, 'A', 'E'],
  ['GLY', 6, -4.433, 2.229, 66.653, 'E', 'G'],
  ['PRO', 7, -5.655, 2.886, 63.078, 'A', 'P'],
  ['ASN', 8, -7.692, 5.999, 63.9, 'A', 'N'],
  ['PHE', 9, -6.641, 6.866, 67.471, 'E', 'F'],
  ['TYR', 10, -3.678, 7.885, 69.649, 'E', 'Y'],
  ['VAL', 11, -4.072, 8.152, 73.41, 'E', 'V'],
  ['PRO', 12, -1.502, 9.047, 76.077, 'E', 'P'],
  ['PHE', 13, -1.827, 6.111, 78.481, 'A', 'F'],
  ['SER', 14, 0.011, 2.815, 78.839, 'E', 'S'],
  ['ASN', 15, -1.953, -0.448, 78.445, 'A', 'N'],
  ['LYS', 16, 0.749, -1.879, 80.75, 'A', 'K'],
  ['THR', 17, -2.053, -1.455, 83.307, 'A', 'T'],
  ['GLY', 18, -4.331, -3.605, 81.088, 'A', 'G'],
  ['VAL', 19, -7.167, -1.066, 80.887, 'A', 'V'],
  ['VAL', 20, -6.61, 1.066, 77.746, 'E', 

In [18]:
accessible_aa = sep_access_aa(all_CA)
hydro_tt = hydro_count (accessible_aa )

In [19]:

num_points = 100+2  # Nombre de points sur la demi-sphère
radius = 5.0  # Rayon de la demi-sphère
les_points = saff_kuijlaars_half_sphere(num_points, radius,centre_de_masse)

In [121]:
centre_de_masse

[-8.681831934831866, 30.219536473326407, 66.80707300790084]

In [20]:
# calcul des vecteur normaux et des plans 
normal_vector = calc_vecteur_normal(les_points,centre_de_masse)
cartesian_d = calcul_eq_d_plan_start(normal_vector,centre_de_masse)

In [21]:
# bouge les plans
maximise_all = max_hydroph(normal_vector,cartesian_d,accessible_aa,hydro_tt)

i0
je suis en bas
i1
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
i2
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
je suis en bas
j

In [22]:
# select meilleur 
les_best1 = best_plane(maximise_all,normal_vector)

In [23]:
lo_max = maximise_hydrophobicite_plane(les_best1,hydro_tt)

1.205128205128205
1.205128205128205
1.217948717948718
 value : 1.217948717948718
1.217948717948718
1.2307692307692308
 value : 1.2307692307692308
1.2307692307692308
1.2307692307692308
1.2307692307692308
1.2307692307692308
1.2307692307692308
1.2307692307692308
1.2307692307692308
1.2435897435897436
 value : 1.2435897435897436
1.2564102564102564
 value : 1.2564102564102564
1.2564102564102564
1.2692307692307692
 value : 1.2692307692307692
1.282051282051282
 value : 1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.282051282051282
1.294871794871795
 value : 1.294871794871795
1.294871794871795
1.294871794871795
1.3076923076923077
 value : 1.3076923076923077
1.3076923076923077
1.3076923076923077
1.3076923076923077
1.3076923076923077
1.3205128205128205
 value : 1.3205128205128205
1.3205128205128205
1.3333333333

In [24]:
lo_max

[[4.112159012800088,
  -0.8225930827615642,
  2.722772277227719,
  98.659,
  -38.34100000000001,
  1.6153846153846154]]

In [29]:
membrane_represention(lo_max,centre_de_masse)

Points générés et enregistrés dans 'membrane_representation0.pdb


In [28]:
import numpy as np

def point_plan(dico_plan1, dico_plan2 ):
    A = dico_plan1['A']
    B = dico_plan1['B']
    C = dico_plan1['C']
    d1 = dico_plan1['D']
    d2 = dico_plan2['D']
    
    x1 = (-d1*A) /(A**2 + B**2 + C**2 )
    y1 = (-d1*B) /(A**2 + B**2 + C**2 )
    z1 = (-d1*C) /(A**2 + B**2 + C**2 )

    x2 = (-d2*A) /(A**2 + B**2 + C**2 )
    y2 = (-d2*B) /(A**2 + B**2 + C**2 )
    z2 = (-d2*C) /(A**2 + B**2 + C**2 )

    return[[x1,y1,z1],[x2,y2,z2]]
    
def membrane_represention (liste_plan,centre_de_masse) : 
    for i in range (0,len(liste_plan)) :
        ax = liste_plan[i][0]
        by = liste_plan[i][1]
        cz = liste_plan[i][2]
        d1 = liste_plan[i][3]
        d2 = liste_plan[i][4]
        plane1 = {'A': ax , 'B': by, 'C': cz , 'D': d1 }
        plane2 = {'A': ax , 'B': by, 'C': cz , 'D': d2 }

        points = point_plan(plane1,plane2)
    

    # Exportez les coordonnées des points vers un fichier PDB
        nom_fichier = f"membrane_representation{i}.pdb"
        with open(nom_fichier, 'w') as f:
            for j in range (0,2):
                    f.write(f"ATOM  {i+1:4}  CA  MEM M   1    {points[j][0]:8.3f}{points[j][1]:8.3f}{points[j][2]:8.3f}\n")

        print(f"Points générés et enregistrés dans '{nom_fichier}")

In [42]:
import numpy as np

# Définir l'équation du plan sous forme ax + by + cz + d = 0
a, b, c, d = 4.11, -0.82, 2.72, 98.66

# Coordonnées du point à projeter
x_point, y_point, z_point = -8.68183193, 30.21953647, 66.80707301

# Calculer la distance entre le point et le plan
distance = (a * x_point + b * y_point + c * z_point + d) / np.sqrt(a**2 + b**2 + c**2)

# Calculer les coordonnées du projeté orthogonal
x_proj = x_point - distance * a
y_proj = y_point - distance * b
z_proj = z_point - distance * c

# Afficher les coordonnées du projeté orthogonal
print("Coordonnées du projeté orthogonal :", (x_proj, y_proj, z_proj))
projette_plan1 = [-10.98939607962352, 30.927220079213345, 62.87850595121286]

Coordonnées du projeté orthogonal : (-189.58450667838773, 66.31204092101653, -52.91440516898167)


In [33]:
import numpy as np

# Définir l'équation du plan sous forme ax + by + cz + d = 0
a, b, c, d2 = 4.11, -0.82, 2.72, -38.34

# Coordonnées du point à projeter
x_point, y_point, z_point = -8.68183193, 30.21953647, 66.80707301

# Calculer la distance entre le point et le plan
distance2 = (a * x_point + b * y_point + c * z_point + d2) / np.sqrt(a**2 + b**2 + c**2)

# Calculer les coordonnées du projeté orthogonal
x_proj2 = x_point - distance2 * a
y_proj2 = y_point - distance2 * b
z_proj2 = z_point - distance2 * c

# Afficher les coordonnées du projeté orthogonal
print("Coordonnées du projeté orthogonal :", (x_proj2, y_proj2, z_proj2))
projette_plan2 = [x_proj2,y_proj2,z_proj2]

Coordonnées du projeté orthogonal : (-76.8868539734045, 43.82735108693228, 21.66895623188315)


In [43]:
nom_fichier ="testpt.pdb"
with open(nom_fichier, 'w') as f:
    for j in range (0,3):
        f.write(f"ATOM  {j+1:4}  CA  MEM M   1    {points[j][0]:8.3f}{points[j][1]:8.3f}{points[j][2]:8.3f}\n")

    print(f"Points générés et enregistrés dans '{nom_fichier}")

Points générés et enregistrés dans 'testpt.pdb


In [35]:
points = [[-8.68183193, 30.21953647, 66.80707301],projette_plan1,projette_plan2]

In [113]:
center_of_mass

[-8.68183193, 30.21953647, 66.80707301]

In [29]:
# Visualisation en 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extraire les coordonnées x, y, z de la liste de points
x_coords, y_coords, z_coords = zip(*sphere_points)

# Afficher les points en 3D
ax.scatter(x_coords, y_coords, z_coords, c='b', marker='o')

# Paramètres de l'axe
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Afficher le graphe
plt.show()

In [35]:
import matplotlib.pyplot as plt

# Visualisation en 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extraire les coordonnées x, y, z de la liste de points
x_coords, y_coords, z_coords = zip(*sphere_points)

# Afficher les points d'intersection en 3D en bleu
ax.scatter(x_coords, y_coords, z_coords, c='b', marker='o', label='Points d\'intersection')

# Coordonnées du centre de masse
cm_x, cm_y, cm_z = -8.68183193, 30.21953647, 66.80707301

# Afficher le centre de masse en rouge
ax.scatter(cm_x, cm_y, cm_z, c='r', marker='o', label='Centre de masse')

# Paramètres de l'axe
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Afficher la légende
ax.legend()

# Afficher le graphe
plt.show()

In [43]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Coordonnées du centre de masse
cm_x, cm_y, cm_z = -8.68183193, 30.21953647, 66.80707301

# Visualisation en 3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Extraire les coordonnées x, y, z de la liste de points
x_coords, y_coords, z_coords = zip(*sphere_points)

# Afficher les points d'intersection en 3D en bleu
ax.scatter(x_coords, y_coords, z_coords, c='b', marker='o', label='Points d\'intersection')

# Afficher le centre de masse en rouge
ax.scatter(cm_x, cm_y, cm_z, c='r', marker='o', label='Centre de masse')

# Afficher les vecteurs entre le centre de masse et les points d'intersection
for x, y, z in zip(x_coords, y_coords, z_coords):
    ax.quiver(cm_x, cm_y, cm_z, x - cm_x, y - cm_y, z - cm_z, color='g', length=100, normalize=True)

# Paramètres de l'axe
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Afficher la légende
ax.legend()

# Afficher le graphe
plt.show()

In [45]:
from Bio.PDB import PDBParser, PDBIO, Atom, Structure
import numpy as np

# 1. Charger les coordonnées des atomes de la protéine à partir d'un fichier PDB
pdb_file = "protein1_only.pdb"
parser = PDBParser()
structure = parser.get_structure("protéine", pdb_file)

# 2. Définir les équations des deux plans (remplacez les valeurs par les vôtres)
plan1 = (4.11, -0.82, 2.72, 98.66)  # Équation du premier plan
plan2 = (4.11, -0.82, 2.72, -38.34)  # Équation du deuxième plan

# 3. Calculer les points d'intersection entre les plans et la protéine
intersection_points = []

for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                atom_coords = atom.get_coord()
                x, y, z = atom_coords
                if (4.11 * x +  -0.82 * y +  2.72 * z + 98.66 == 0) and (4.11 * x +  -0.82 * y + 2.72 * z + -38.34 == 0):
                    intersection_points.append(atom_coords)

# 4. Créer un fichier PDB pour les sphères représentant les points d'intersection
intersection_structure = Structure.Structure("intersection")
intersection_model = Structure.Model(0)
intersection_structure.add(intersection_model)

for point_coords in intersection_points:
    atom = Atom.Atom("CA", point_coords, 0, 1, "", "X", 0, "C")
    intersection_model.add(atom)

output_pdb = "intersection_points.pdb"
io = PDBIO()
io.set_structure(intersection_structure)
io.save(output_pdb)

print(f"Points d'intersection enregistrés dans {output_pdb}")


AttributeError: module 'Bio.PDB.Structure' has no attribute 'Model'

In [46]:
from Bio.PDB import PDBParser, PDBIO, Atom, Structure
import numpy as np

# 1. Charger les coordonnées des atomes de la protéine à partir d'un fichier PDB
pdb_file = "protein1_only.pdb"
parser = PDBParser()
structure = parser.get_structure("protéine", pdb_file)

# 2. Définir les équations des deux plans (remplacez les valeurs par les vôtres)
plan1 = (4.11, -0.82, 2.72, 98.66)  # Équation du premier plan
plan2 = (4.11, -0.82, 2.72, -38.34)  # Équation du deuxième plan

# 3. Calculer les points d'intersection entre les plans et la protéine
intersection_points = []

for model in structure:
    for chain in model:
        for residue in chain:
            for atom in residue:
                atom_coords = atom.get_coord()
                x, y, z = atom_coords
                if (4.11 * x +  -0.82 * y +  2.72 * z + 98.66 == 0) and (4.11 * x +  -0.82 * y + 2.72 * z + -38.34 == 0):
                    intersection_points.append(atom_coords)

In [47]:
atom_coords

array([-19.241,  61.586,  74.466], dtype=float32)

In [2]:
nbconvert --to pythonconfig_template.ipynb

SyntaxError: invalid syntax (3985629202.py, line 1)